In [1]:
import jax.numpy as jnp
import numpy as np
import time
import chex
import jax
import gymnasium as gym
import exciting_environments as excenv
import random
import matplotlib.pyplot as plt
import diffrax

### Simulate Ahead

In [12]:
env=excenv.Pendulum(batch_size=5)

In [13]:
acts=jnp.full((5,8),jnp.array([1,1,1,1,1,1,1,1])*10)[:,:,None]

In [14]:
states=env.init_state()
obs,rew,truncated,terminated,last_state=env.vmap_sim_ahead(states,
                   acts,
                   env.tau,
                   env.tau
                   )
obs[0]

Array([[-1.0000000e+00,  0.0000000e+00],
       [-1.0000000e+00,  5.0000002e-04],
       [-9.9999982e-01,  1.0000000e-03],
       [-9.9999952e-01,  1.4999999e-03],
       [-9.9999911e-01,  1.9999999e-03],
       [-9.9999845e-01,  2.4999995e-03],
       [-9.9999774e-01,  2.9999993e-03],
       [-9.9999678e-01,  3.4999989e-03],
       [-9.9999559e-01,  3.9999983e-03]], dtype=float32, weak_type=True)

In [15]:
state=env.init_state()
for _ in range(acts.shape[1]):
    obs,state=env.vmap_step(state=state,action=10*jnp.ones(5).reshape(-1,1))
    print(obs[0,:])

[-1.e+00  5.e-04]
[-0.9999998  0.001    ]
[-0.9999995  0.0015   ]
[-0.9999991  0.002    ]
[-0.99999845  0.0025    ]
[-0.99999774  0.003     ]
[-0.9999967  0.0035   ]
[-0.9999956  0.004    ]
